In [ ]:
#%matplotlib notebook
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import importlib
import kk.tools
import kk.uitools
from kk.dggs import DGGS
#importlib.reload(kk.tools)


In [ ]:
dg = DGGS()

def cell_bounds(addr):
    tr, (maxW, maxH) = dg.pixel_coord_transform(addr, 1,1, native=True, no_offset=True)
    x_min, y_min = tr(0,0)
    x_max, y_max = tr(1,1)
    return (x_min, x_max, y_min, y_max)

def cell_center(addr):
    tr, (maxW, maxH) = dg.pixel_coord_transform(addr, 1,1, native=True)
    return tr(0,0)


def merge_extents(e1, e2):
    if e1 is None:
        return e2
    
    return [min(e1[0], e2[0]), max(e1[1], e2[1]),
            min(e1[2], e2[2]), max(e1[3], e2[3])]

def hide_axis(ax):
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.axis('off')
    
def plot_all(data, south_square=0, north_square=0, ax=None):
    dg = DGGS()
    dh = dg.mk_display_helper(south_square=south_square, north_square=north_square)
    
    if ax is None:
        fig = plt.figure()
        ax = fig.add_axes([0,0,1,1])
    else:
        fig = None
        
    extents = None

    for addr, ds in data.items():
        
        im = np.dstack([ds.red.values, ds.green.values, ds.blue.values] )
        im, ee = dh(addr, im)
        extents = merge_extents(extents, ee)
        ax.imshow(im, extent=ee)

    ax.set_xlim(*extents[:2])
    ax.set_ylim(*extents[2:])
    return fig

In [ ]:
dd = kk.tools.h5_load('earth.h5')

In [ ]:
fig0 = plot_all(dd, south_square=0)
fig23 = plot_all(dd, south_square=3, north_square=2)

for ax in fig0.axes:
    hide_axis(ax)
for ax in fig23.axes:
    hide_axis(ax)

In [ ]:
fig0.savefig('dggs_earth_00.png', dpi=300)
fig23.savefig('dggs_earth_23.png', dpi=300)

In [ ]:
#!open dggs_earth_23.png

In [ ]:
fig0 = plot_all(dd)
for ax in fig0.axes:
    hide_axis(ax)

for c in 'SNPQR':
    kk.uitools.add_cell_plot(c, 'y-', linewidth=2, alpha=0.3, native=True)